In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments, Trainer, default_data_collator, EarlyStoppingCallback
from datasets import Dataset

/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda:0


In [3]:
# Load the pre-trained BERT model and tokenizer
#mod = "mistralai/Mistral-7B-v0.1"
model_name = "/home/patrick.araujo/llama2/llama/llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████████████| 3/3 [00:02<00:00,  1.30it/s]


In [5]:
text = "Hi. How are you?"

In [6]:
text

'Hi. How are you?'

In [7]:
encoded_text = tokenizer(text)["input_ids"]

In [8]:
encoded_text

[1, 6324, 29889, 1128, 526, 366, 29973]

In [9]:
decoded_text = tokenizer.decode(encoded_text)
print("Decoded tokens back into text: ", decoded_text)

Decoded tokens back into text:  <s> Hi. How are you?


In [10]:
list_texts = ["Hi, how are you?", "I'm good", "Yes"]
encoded_texts = tokenizer(list_texts)
print("Encoded several texts: ", encoded_texts["input_ids"])

Encoded several texts:  [[1, 6324, 29892, 920, 526, 366, 29973], [1, 306, 29915, 29885, 1781], [1, 3869]]


In [11]:
tokenizer.pad_token = tokenizer.eos_token 
encoded_texts_longest = tokenizer(list_texts, padding=True)
print("Using padding: ", encoded_texts_longest["input_ids"])

Using padding:  [[1, 6324, 29892, 920, 526, 366, 29973], [2, 2, 1, 306, 29915, 29885, 1781], [2, 2, 2, 2, 2, 1, 3869]]


In [12]:
encoded_texts_truncation = tokenizer(list_texts, max_length=3, truncation=True)
print("Using truncation: ", encoded_texts_truncation["input_ids"])

Using truncation:  [[1, 6324, 29892], [1, 306, 29915], [1, 3869]]


In [13]:
tokenizer.truncation_side = "left"
encoded_texts_truncation_left = tokenizer(list_texts, max_length=3, truncation=True)
print("Using left-side truncation: ", encoded_texts_truncation_left["input_ids"])

Using left-side truncation:  [[1, 366, 29973], [1, 29885, 1781], [1, 3869]]


In [14]:
encoded_texts_both = tokenizer(list_texts, max_length=3, truncation=True, padding=True)
print("Using both padding and truncation: ", encoded_texts_both["input_ids"])

Using both padding and truncation:  [[1, 366, 29973], [1, 29885, 1781], [2, 1, 3869]]


In [38]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = 'right'

In [15]:
dataset = pd.read_csv('/home/patrick.araujo/llama2/datasets/balanced_output_train.csv')

In [16]:
dataset

,level_0,index,Unnamed: 0,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment,lengthContent,Language
0,1007,1527,1527,7d6a4b7f-0f17-47c6-8010-3187dd1c86a7,I've been using my visa gift card. All the inf...,1,2,26.23.4.100,2023-12-28 21:20:56,NaN,NaN,26.23.4.100,0,224,en
1,8622,12941,12941,227c23e5-e178-4d69-bb99-3fe1445dc035,the PRIME PRICE IS WAY TO HIGH!!,5,0,26.19.2.100,2023-10-02 12:09:02,NaN,NaN,26.19.2.100,2,32,en
2,7774,11420,11420,bc748a49-bbd9-4773-a63f-f0950ad66310,easy to use and fast free delivery,5,0,26.20.0.100,2023-10-25 03:46:34,NaN,NaN,26.20.0.100,2,34,en
3,5478,8135,8135,0b2c1d90-3026-427a-bea9-918d1e067e8f,The Shopping On Amazon Is The Greatest Of All....,5,0,NaN,2023-11-12 01:54:20,NaN,NaN,NaN,2,111,en
4,13393,19072,19072,bee9613a-352d-4328-abaa-986316fe788d,I am rarely not satisfied seems I'm always sat...,4,0,24.22.0.100,2023-01-10 09:30:07,NaN,NaN,24.22.0.100,2,64,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9307,7444,10822,10822,e95c926d-6510-4b97-8ad7-7b11885a46a7,Amazon is always my go to for everything!!,5,0,26.21.0.100,2023-11-03 01:06:14,NaN,NaN,26.21.0.100,2,42,en
9308,17040,24320,24320,f26c2ae2-6868-49ab-9845-963ce24f14f8,This is a great app but I didn't get my neckla...,3,1,22.14.0.100,2022-06-01 13:35:57,NaN,NaN,22.14.0.100,1,240,en
9309,14542,20557,20557,cd102f54-3164-494c-8287-b2064c82401b,11-3-22- I have 107 S&S items. It is impossibl...,4,9,24.20.2.100,2022-11-03 11:59:23,NaN,NaN,24.20.2.100,2,320,en
9310,16186,22964,22964,d8372268-d77e-4666-a2dd-166decb97443,"App sucks! Glitchy, slow, hurts my eyes cuz im...",3,0,24.12.6.100,2022-07-21 14:35:03,NaN,NaN,24.12.6.100,1,231,en


In [17]:
dataset.shape

(9312, 15)

In [18]:
# Create train and validation datasets
comments = dataset[['content', 'sentiment']]
train_data, val_data = train_test_split(comments, test_size=0.2, random_state=42)

In [19]:
print(train_data.shape)
print(val_data.shape)

(7449, 2)
(1863, 2)


In [20]:
def returnSentiment(sentiment):
    if sentiment == 0:
        return 'negative'
    elif sentiment == 1:
        return 'neutral'
    elif sentiment == 2:
        return 'positive'

def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the comment enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["content"]}] = {returnSentiment(data_point["sentiment"])}
            """.strip()

In [21]:
def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the comment enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["content"]}] = """.strip()

In [22]:
X_train = pd.DataFrame(train_data.apply(generate_prompt, axis=1), columns=['text'])

In [23]:
print(X_train.iloc[0]['text'])

Analyze the sentiment of the comment enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [Usually love the app but lately if I try to use the baby registry and remove things from the registry the app freezes and I have to close it and start over. It's super annoying please fix it.] = neutral


In [24]:
X_train

,text
503,Analyze the sentiment of the comment enclosed ...
5813,Analyze the sentiment of the comment enclosed ...
274,Analyze the sentiment of the comment enclosed ...
1345,Analyze the sentiment of the comment enclosed ...
5057,Analyze the sentiment of the comment enclosed ...
...,...
5734,Analyze the sentiment of the comment enclosed ...
5191,Analyze the sentiment of the comment enclosed ...
5390,Analyze the sentiment of the comment enclosed ...
860,Analyze the sentiment of the comment enclosed ...


In [25]:
X_eval = pd.DataFrame(val_data.apply(generate_test_prompt, axis=1), columns=['text'])

In [26]:
print(X_eval.iloc[0]['text'])

Analyze the sentiment of the comment enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [Amazon shopping app doesn't allow user to clear the search history. No option to clear searching like eBay or other shopping app.] =


In [27]:
X_eval

,text
8638,Analyze the sentiment of the comment enclosed ...
131,Analyze the sentiment of the comment enclosed ...
3509,Analyze the sentiment of the comment enclosed ...
4401,Analyze the sentiment of the comment enclosed ...
3095,Analyze the sentiment of the comment enclosed ...
...,...
6657,Analyze the sentiment of the comment enclosed ...
4271,Analyze the sentiment of the comment enclosed ...
2404,Analyze the sentiment of the comment enclosed ...
6603,Analyze the sentiment of the comment enclosed ...


In [28]:
train_info = Dataset.from_pandas(X_train)

In [29]:
train_info

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 7449
})

In [30]:
eval_info = Dataset.from_pandas(X_eval)

In [31]:
eval_info

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 1863
})

In [32]:
dataset_test = pd.read_csv('/home/patrick.araujo/llama2/datasets/balanced_output_val.csv')

In [33]:
dataset_test

,level_0,index,Unnamed: 0,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment,lengthContent,Language
0,14500,20500,20500,2224e4ab-4a0b-4282-b6b8-b6bd5257013e,I felt like it was misleading and it's wording...,3,0,24.20.2.100,2022-11-06 01:03:17,NaN,NaN,24.20.2.100,1,500,en
1,731,1111,1111,21802655-da6a-41a6-b722-cf1c556ff9fe,Locked out of my account years ago. Stop bothe...,1,0,22.14.0.100,2023-12-31 18:14:04,NaN,NaN,22.14.0.100,0,54,en
2,11211,16310,16310,8e810875-04bf-43e8-856f-f27c91297c32,I like the offers and special payment plans!,4,0,26.11.0.100,2023-06-07 09:03:35,NaN,NaN,26.11.0.100,2,44,en
3,7960,11767,11767,3e469da5-b7d4-4bf6-94c9-82682b14196c,App is okay one thing it needs is a place wher...,4,1,26.19.4.100,2023-10-19 18:06:51,NaN,NaN,26.19.4.100,2,361,en
4,14144,20043,20043,5ef43a6f-4974-40fe-89fb-5ace978806ab,The option to save item to shopping list is no...,3,65,24.21.4.100,2022-11-27 17:33:28,NaN,NaN,24.21.4.100,1,458,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,5671,8451,8451,f6c6b9d8-bc59-4605-9e52-ac9fa030ae83,"Great prices, but Community standards do not a...",2,1,26.21.0.100,2023-11-09 21:34:30,NaN,NaN,26.21.0.100,0,69,en
2324,2661,3858,3858,240a8243-bde6-454e-a1d9-3b494ba1d509,Not Working...unable to subscribe to Amazon Pr...,1,0,26.23.0.100,2023-12-11 05:38:35,NaN,NaN,26.23.0.100,0,50,en
2325,7593,11085,11085,418e4d12-28de-48c3-a07a-017a41b4c384,best options to buy when you are working home ...,5,0,26.21.0.100,2023-10-30 09:37:11,NaN,NaN,26.21.0.100,2,113,en
2326,3975,5793,5793,ccab6a92-43ae-46d7-a694-b3d4d8d9cae0,The most annoying thing about Amazon is that t...,3,0,26.22.0.100,2023-11-26 20:24:00,NaN,NaN,26.22.0.100,1,186,en


In [34]:
y_true = dataset_test['sentiment']

In [35]:
y_true

0       1
1       0
2       2
3       2
4       1
       ..
2323    0
2324    0
2325    2
2326    1
2327    1
Name: sentiment, Length: 2328, dtype: int64

In [36]:
X_test = pd.DataFrame(dataset_test.apply(generate_test_prompt, axis=1), columns=['text'])

In [37]:
X_test

,text
0,Analyze the sentiment of the comment enclosed ...
1,Analyze the sentiment of the comment enclosed ...
2,Analyze the sentiment of the comment enclosed ...
3,Analyze the sentiment of the comment enclosed ...
4,Analyze the sentiment of the comment enclosed ...
...,...
2323,Analyze the sentiment of the comment enclosed ...
2324,Analyze the sentiment of the comment enclosed ...
2325,Analyze the sentiment of the comment enclosed ...
2326,Analyze the sentiment of the comment enclosed ...


In [49]:
y_pred = []
for i in range(len(X_test)):
    prompt = X_test.iloc[i]["text"]
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 1, temperature = 0.0)
    result = pipe(prompt)
    answer = result[0]['generated_text'].split("=")[-1]
    if "positive" in answer:
        y_pred.append("positive")
    elif "negative" in answer:
        y_pred.append("negative")
    elif "neutral" in answer:
        y_pred.append("neutral")
    else:
        y_pred.append("none")
    print(f'{str(i)}: {result}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0: [{'generated_text': 'Analyze the sentiment of the comment enclosed in square brackets, \n            determine if it is positive, neutral, or negative, and return the answer as \n            the corresponding sentiment label "positive" or "neutral" or "negative".\n\n            [I felt like it was misleading and it\'s wording you guys say comforter blue red comforters set with sheets not blue red cover or comforter cover and now I\'m stuck with something I can\'t use I don\'t even know what comforter with fit that are go in it or what\'s best to get and I feel like if you need guys know that and can facilitate it would think that you can reconcile this with sending me the comforter that goes inside of that I would have never paid $60 for cover when I paid less for the sheets] = "'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1: [{'generated_text': 'Analyze the sentiment of the comment enclosed in square brackets, \n            determine if it is positive, neutral, or negative, and return the answer as \n            the corresponding sentiment label "positive" or "neutral" or "negative".\n\n            [Locked out of my account years ago. Stop bothering me.] = "'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

2324: [{'generated_text': 'Analyze the sentiment of the comment enclosed in square brackets, \n            determine if it is positive, neutral, or negative, and return the answer as \n            the corresponding sentiment label "positive" or "neutral" or "negative".\n\n            [Not Working...unable to subscribe to Amazon Prime!] = "'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2325: [{'generated_text': 'Analyze the sentiment of the comment enclosed in square brackets, \n            determine if it is positive, neutral, or negative, and return the answer as \n            the corresponding sentiment label "positive" or "neutral" or "negative".\n\n            [best options to buy when you are working home with a baby and have no time to shop what you need ! Love Amazon ❤️] = "'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2326: [{'generated_text': 'Analyze the sentiment of the comment enclosed in square brackets, \n            determine if it is positive, neutral, or negative, and return the answer as \n            the corresponding sentiment label "positive" or "neutral" or "negative".\n\n            [The most annoying thing about Amazon is that they don\'t always charge you when you place an order, and it can be frustrating at times. This never happens with Ebay. Get with the program.] = "'}]
2327: [{'generated_text': 'Analyze the sentiment of the comment enclosed in square brackets, \n            determine if it is positive, neutral, or negative, and return the answer as \n            the corresponding sentiment label "positive" or "neutral" or "negative".\n\n            [They take too long to deliver plus they need to come on] = "'}]


In [ ]:
# Define a custom function to map scores to sentiment
def map_num_to_sentiment(score):
    if score == 0:
        return 'negative'
    elif score == 1:
        return 'neutral'
    elif score == 2:
        return 'positive'
        
# Apply the custom function to create a new 'sentiment' column
y_true_labels = y_true.apply(map_num_to_sentiment)

In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming you have true labels (y_true) and predicted labels (y_pred)
accuracy = accuracy_score(y_true_labels, y_pred)
precision_macro = precision_score(y_true_labels, y_pred, average='macro')
recall = recall_score(y_true_labels, y_pred, average='macro')
f1 = f1_score(y_true_labels, y_pred, average='macro')
conf_matrix = confusion_matrix(y_true_labels, y_pred)

print("Accuracy:", accuracy)
print("Precision (Macro):", precision_macro)
print("Recall (Macro):", recall)
print("F1 Score (Macro):", f1)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.02190721649484536
Precision (Macro): 0.40909090909090906
Recall (Macro): 0.01643041237113402
F1 Score (Macro): 0.031356245380635625
Confusion Matrix:
[[ 12   0 762   2]
 [  2   0 771   3]
 [  0   0   0   0]
 [  2   0 735  39]]


/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [54]:
batch_size = 8

In [58]:
# Define training arguments with early stopping
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir="./llama2_model",
    evaluation_strategy="steps",
    eval_steps=100,  # Adjust as needed
    save_total_limit=2,  # Adjust as needed. Only last 2 models are saved. Older ones are deleted.
    num_train_epochs=3,  # Adjust as needed
    save_steps=500,  # Adjust as needed
    logging_dir="./logs",
    metric_for_best_model="eval_loss",  # Use validation loss for early stopping
    load_best_model_at_end=True
)

In [59]:
def compute_metrics(p):
  pred, labels = p
  pred = np.argmax(pred, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred, average='macro')
  precision = precision_score(y_true=labels, y_pred=pred, average='macro')
  f1 = f1_score(y_true=labels, y_pred=pred, average='macro')  # Add line for F1 score
  return  {
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1': f1  # Include F1 score in the dictionary
          }

In [72]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_info,
    eval_dataset=eval_info,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [73]:
trainer.train()

IndexError: Invalid key: 7429 is out of bounds for size 0